In [9]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, LinearColorMapper, LogColorMapper
from bokeh.transform import factor_cmap
from bokeh.palettes import Colorblind, Magma256

output_notebook()
palette = Colorblind[8]
palette_gradient = Magma256

Loading BokehJS ...

In [10]:
df_iterations = pd.read_csv("../results.renaissance.test.csv")

In [11]:
df_iterations["iteration_start_ns"] = df_iterations.groupby(["benchmark", "pair"])[
    "iteration_time_ns"
].transform(pd.Series.cumsum)
df_iterations["iteration_start_ns"] = df_iterations.groupby(["benchmark", "pair"])[
    "iteration_start_ns"
].shift(1, fill_value=0)
df_iterations["iteration_start_ns"] = (
    df_iterations["iteration_start_ns"] + df_iterations["epoch_start_ms"] * 1000
)

df_iterations["iteration_end_ns"] = (
    df_iterations["iteration_start_ns"] + df_iterations["iteration_time_ns"]
)

df_iterations.describe()

,runid,iteration,epoch_start_ms,iteration_time_ns,machine,provider,iteration_start_ns,iteration_end_ns
count,40.00000,40.000000,4.000000e+01,4.000000e+01,0.0,0.0,4.000000e+01,4.000000e+01
mean,0.50000,4.500000,1.647775e+12,2.609017e+09,NaN,NaN,1.647804e+15,1.647807e+15
std,0.50637,2.908872,2.350872e+04,1.934521e+09,NaN,NaN,1.475596e+10,1.397709e+10
min,0.00000,0.000000,1.647775e+12,1.535864e+09,NaN,NaN,1.647775e+15,1.647784e+15
25%,0.00000,2.000000,1.647775e+12,1.753671e+09,NaN,NaN,1.647793e+15,1.647795e+15
50%,0.50000,4.500000,1.647775e+12,1.957464e+09,NaN,NaN,1.647802e+15,1.647806e+15
75%,1.00000,7.000000,1.647775e+12,2.348786e+09,NaN,NaN,1.647817e+15,1.647819e+15
max,1.00000,9.000000,1.647775e+12,9.081739e+09,NaN,NaN,1.647826e+15,1.647828e+15


In [12]:
source = ColumnDataSource(df_iterations[df_iterations.iteration < 10])
pair_cmap = factor_cmap(
    "pair", palette=palette, factors=sorted(df_iterations["pair"].unique())
)
p = figure()
p.circle(x="iteration_start_ns", y="iteration", color=pair_cmap, size=9, source=source)
p.cross(x="iteration_end_ns", y="iteration", color=pair_cmap, size=9, source=source)

show(p)

In [13]:
columns_of_interest = [
    "benchmark",
    "pair",
    "runid",
    "iteration",
    "iteration_time_ns",
    "iteration_start_ns",
    "iteration_end_ns",
]
df = df_iterations[columns_of_interest]

# Compute overlap interval
df_overlap = df[df.pair == "A"].merge(
    df[df.pair == "B"], on=["benchmark", "runid"], suffixes=["_A", "_B"], how="inner"
)
df_overlap["overlap_start_ns"] = df_overlap[
    ["iteration_start_ns_A", "iteration_start_ns_B"]
].max(axis=1)
df_overlap["overlap_end_ns"] = df_overlap[
    ["iteration_end_ns_A", "iteration_end_ns_B"]
].min(axis=1)

df_overlap.drop(["pair_A", "pair_B"], axis=1, inplace=True)

# Filter out non overlapping intervals
df_overlap["overlap"] = df_overlap["overlap_start_ns"].lt(df_overlap["overlap_end_ns"])
df_overlap = df_overlap[df_overlap["overlap"] == True]
df_overlap.reset_index(drop=True, inplace=True)

# Compute overlap properties
df_overlap["overlap_size"] = (
    df_overlap["overlap_end_ns"] - df_overlap["overlap_start_ns"]
)
df_overlap["overlap_proportion_A"] = (
    df_overlap["overlap_size"] / df_overlap["iteration_time_ns_A"]
)
df_overlap["overlap_proportion_B"] = (
    df_overlap["overlap_size"] / df_overlap["iteration_time_ns_B"]
)
df_overlap["iteration_difference_time_ns"] = (
    df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"]
)
df_overlap["iteration_difference_time_ns_size_weighted"] = (
    df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"]
) * df_overlap["overlap_size"]
df_overlap["iteration_difference_time_ns_proportion_weighted"] = (
    df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"]
) * df_overlap["overlap_proportion_A"]
df_overlap["iteration_difference_time_ns_scaled"] = (
    df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"]
) / df_overlap["iteration_time_ns_A"]
df_overlap["iteration_difference_time_ns_scaled_size_weighted"] = (
    (df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"])
    / df_overlap["iteration_time_ns_A"]
) * df_overlap["overlap_size"]
df_overlap["iteration_difference_time_ns_scaled_proportion_weighted"] = (
    (df_overlap["iteration_time_ns_A"] - df_overlap["iteration_time_ns_B"])
    / df_overlap["iteration_time_ns_A"]
) * df_overlap["overlap_proportion_A"]

df_overlap

,benchmark,runid,iteration_A,iteration_time_ns_A,iteration_start_ns_A,iteration_end_ns_A,iteration_B,iteration_time_ns_B,iteration_start_ns_B,iteration_end_ns_B,...,overlap,overlap_size,overlap_proportion_A,overlap_proportion_B,iteration_difference_time_ns,iteration_difference_time_ns_size_weighted,iteration_difference_time_ns_proportion_weighted,iteration_difference_time_ns_scaled,iteration_difference_time_ns_scaled_size_weighted,iteration_difference_time_ns_scaled_proportion_weighted
0,chi-square,0,0,9052543216,1647774987700000,1647784040243216,0,9081738837,1647774987700000,1647784069438837,...,True,9052543216,1.000000,0.996785,-29195621,-264294620820457136,-2.919562e+07,-0.003225,-2.919562e+07,-0.003225
1,chi-square,0,1,1960601313,1647784040243216,1647786000844529,0,9081738837,1647774987700000,1647784069438837,...,True,29195621,0.014891,0.003215,-7121137524,-207906032239582404,-1.060420e+08,-3.632119,-1.060420e+08,-0.054086
2,chi-square,0,1,1960601313,1647784040243216,1647786000844529,1,2394048981,1647784069438837,1647786463487818,...,True,1931405692,0.985109,0.806753,-433447668,-837163293159326256,-4.269931e+08,-0.221079,-4.269931e+08,-0.217787
3,chi-square,0,2,2594764894,1647786000844529,1647788595609423,1,2394048981,1647784069438837,1647786463487818,...,True,462643289,0.178299,0.193247,200715913,92859870144957857,3.578739e+07,0.077354,3.578739e+07,0.013792
4,chi-square,0,2,2594764894,1647786000844529,1647788595609423,2,2662040185,1647786463487818,1647789125528003,...,True,2132121605,0.821701,0.800935,-67275291,-143439101423762055,-5.528019e+07,-0.025927,-5.528019e+07,-0.021305
5,chi-square,0,3,2469688466,1647788595609423,1647791065297889,2,2662040185,1647786463487818,1647789125528003,...,True,529918580,0.214569,0.199065,-192351719,-101930749793039020,-4.127272e+07,-0.077885,-4.127272e+07,-0.016712
6,chi-square,0,3,2469688466,1647788595609423,1647791065297889,3,2949463436,1647789125528003,1647792074991439,...,True,1939769886,0.785431,0.657669,-479774970,-930653038862553420,-3.768301e+08,-0.194265,-3.768301e+08,-0.152582
7,chi-square,0,4,1941899087,1647791065297889,1647793007196976,3,2949463436,1647789125528003,1647792074991439,...,True,1009693550,0.519952,0.342331,-1007564349,-1017331224395248950,-5.238847e+08,-0.518855,-5.238847e+08,-0.269780
8,chi-square,0,4,1941899087,1647791065297889,1647793007196976,4,1906221669,1647792074991439,1647793981213108,...,True,932205537,0.480048,0.489033,35677418,33258686605463466,1.712689e+07,0.018372,1.712689e+07,0.008820
9,chi-square,0,5,2020537827,1647793007196976,1647795027734803,4,1906221669,1647792074991439,1647793981213108,...,True,974016132,0.482058,0.510967,114316158,111345782040260856,5.510700e+07,0.056577,5.510700e+07,0.027273


In [14]:
df_overlap[
    [
        "iteration_time_ns_A",
        "iteration_time_ns_B",
        "overlap_size",
        "iteration_difference_time_ns",
        "iteration_difference_time_ns_size_weighted",
        "iteration_difference_time_ns_proportion_weighted",
        "iteration_difference_time_ns_scaled_size_weighted",
        "iteration_difference_time_ns_scaled_proportion_weighted",
    ]
].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
iteration_time_ns_A,38.0,2.280175e+09,1.456050e+09,1.535864e+09,1.754025e+09,1.941899e+09,2.099392e+09,9.052543e+09
iteration_time_ns_B,38.0,2.700905e+09,1.962720e+09,1.602075e+09,1.794969e+09,2.024387e+09,2.500059e+09,9.081739e+09
overlap_size,38.0,1.320279e+09,1.630331e+09,2.260885e+07,5.129990e+08,1.031699e+09,1.526829e+09,9.052543e+09
iteration_difference_time_ns,38.0,-4.207303e+08,1.387741e+09,-7.121138e+09,-2.769427e+08,-4.198269e+07,3.432671e+07,3.057078e+08
iteration_difference_time_ns_size_weighted,38.0,-3.034683e+17,9.801497e+17,-5.822087e+18,-2.760628e+17,-4.717705e+16,1.074245e+16,5.093165e+17
iteration_difference_time_ns_proportion_weighted,38.0,-1.453788e+08,4.242980e+08,-2.494790e+09,-1.447159e+08,-2.121571e+07,5.529463e+06,1.515086e+08
iteration_difference_time_ns_scaled_size_weighted,38.0,-1.453788e+08,4.242980e+08,-2.494790e+09,-1.447159e+08,-2.121571e+07,5.529463e+06,1.515086e+08
iteration_difference_time_ns_scaled_proportion_weighted,38.0,-7.026682e-02,1.867112e-01,-1.069029e+00,-7.412223e-02,-5.651569e-03,2.846487e-03,7.113720e-02


In [15]:
source = ColumnDataSource(df_overlap)
p = figure()
p.hbar(
    y="index",
    left="overlap_start_ns",
    right="overlap_end_ns",
    source=source,
    color={
        "field": "overlap_size",
        "transform": LogColorMapper(palette=palette_gradient),
    },
)

show(p)

In [16]:
source = ColumnDataSource(df_overlap)
p = figure()
p.vbar(x="overlap_start_ns", top="overlap_size", source=source)

show(p)